# THE BATTLE OF NEIGHBORHOODS

## 1. Description of the problem and a discussion of the background.

### Problem: Choosing the best neighborhood to live in NY and for me this means having vegetarian/vegan restaurant close, having good coffee shop close and as well having a nice park where for running.



#### I will be moving to New York next year and I'm looking for a neighborhood. The neighboor will have to have at least 3 vegan restaurant in a ratio of 1000 meters, two nice coffee shop to have a good cappuccino there with good rating and a park for jogging there.

#### Let's see if we find a neighborhood like this which matchs the criteria describe above.

#### Questions that we are going to answer using data:
#### 1. Which neighboorhood in New York has at least 3 vegan restaurant?
#### 2. Which neighboorhood in New York has at least two nice coffe shop?
#### 3. Which neighboorhood has a park in New York has?

####  

Why I like New York so much:

The City of New York, usually known as either New York City (NYC), or simply New York (NY), is the most populous city in the United States. With an estimated 2018 population of 8,398,748 distributed over about 302.6 square miles (784 km2), New York is also the most densely populated major city in the United States. Located at the southern tip of the U.S. state of New York, the city is the center of the New York metropolitan area, the largest metropolitan area in the world by urban landmass. With almost 20 million people in its metropolitan statistical area and approximately 23 million in its combined statistical area, it is one of the world's most populous megacities. New York City has been described as the cultural, financial, and media capital of the world, significantly influencing commerce, entertainment, research, technology, education, politics, tourism, art, fashion, and sports. Home to the headquarters of the United Nations, New York is an important center for international diplomacy.

Situated on one of the world's largest natural harbors, New York City is divided into five administrative boroughs, each of which is a separate county of the State of New York. The five boroughs – Brooklyn, Queens, Manhattan, The Bronx, and Staten Island – were consolidated into a single city in 1898. The city and its metropolitan area constitute the premier gateway for legal immigration to the United States. As many as 800 languages are spoken in New York, making it the most linguistically diverse city in the world. New York is home to 3.2 million residents born outside the United States, the largest foreign-born population of any city in the world as of 2016. As of 2019, the New York metropolitan area is estimated to produce a gross metropolitan product (GMP) of $2.0 trillion. If greater New York City were a sovereign state, it would have the 12th highest GDP in the world. New York is home to the highest number of billionaires of any city in the world.


## 2. Description of the data and how it will be used to solve the problem. 

### We will use the Foursquare API to answer the first two question. 
We will divide the first two question in small question and we will plot some partial result.
For the question number 3 we will use de Google Maps API to find out with neighboorhoods has a park for running.

### The data that we will use is located in https://geo.nyu.edu/catalog/nyu_2451_34572
We will answer the 3 main questions above using the Foursquare API to find the Vegatarian/Vegan restaurants in New York and the Google Maps API to find the parks